In [ ]:
# data from here: https://www.kaggle.com/cnic92/200-financial-indicators-of-us-stocks-20142018?select=2015_Financial_Data.csv

In [130]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

In [131]:
df1v1 = pd.read_csv(os.path.join("..", "2014_Financial_Data.csv"))
df2v1 = pd.read_csv(os.path.join("..", "2015_Financial_Data.csv"))

In [134]:
# note: hypothesized financial indicators listed below
hypothesized_indicators = ['Revenue Growth', 'Net Income', 'EPS', 'EBITDA Margin', 'priceToSalesRatio', 'priceEarningsRatio', 'priceToFreeCashFlowsRatio', 'Debt to Equity', 
                          'ROIC', 'Sector']

def makeAdjustments(df, df2, year):
    # filter for relevant columns    
    tickers = df['Unnamed: 0']
    df = df[['Revenue Growth', 'Net Income', 'EPS', 'EBITDA Margin', 'priceToSalesRatio', 'priceEarningsRatio', 'priceToFreeCashFlowsRatio', 'Debt to Equity', 'ROIC', 'Sector',
              str(year+1) + ' PRICE VAR [%]']]
    df = pd.concat([df, tickers], axis=1)
    df = df.rename(columns={'Unnamed: 0': 'Ticker',
                           'Revenue Growth': str(year) + ' Revenue Growth'})
    
    # drop infinity+NaN and reset index
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.dropna()
    df = df.reset_index()
    df.drop(['index'], axis=1, inplace=True)
    
    # create next year returns dataset with next year's returns
    returns = df2[['Unnamed: 0', str(year+2) + ' PRICE VAR [%]']]
    returns = returns.rename(columns={'Unnamed: 0': 'Ticker'})
    df = df.merge(returns, left_on = 'Ticker', right_on = 'Ticker', how = 'left')
    
    # create increase/decrease variable -- increase = 1, decrease = 0
    temp = []
    for i in range(len(df)):
        yr_return = df[str(year+2) + ' PRICE VAR [%]'][i]
        if yr_return > 0 and yr_return < 10:
            temp.append('1')
        elif yr_return > 10:
            temp.append('2')
        else:
            temp.append('0')
    df['Target'] = temp
    
    # create dummy variables for sector categorical variable
    cat_variables = df[['Sector']]
    cat_dummies = pd.get_dummies(cat_variables)
    df.drop(['Sector'], axis=1, inplace=True)
    df = pd.concat([df, cat_dummies], axis=1)
    
    df = df.drop(['Ticker'], axis=1)
    
    # change datatypes??????
    
    return df    

In [135]:
df1v2 = makeAdjustments(df1v1, df2v1, 2014)

In [136]:
def runRandomForest(df, year):
    y = df['Target']
    
    data = df.drop(['Target', str(year+1) + ' PRICE VAR [%]', str(year+2) + ' PRICE VAR [%]', 'Ticker'], axis=1)
    feature_names = data.columns

    X = df[feature_names]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#     # Create the model with 100 trees
#     model = RandomForestClassifier(n_estimators=100, 
#                                    bootstrap = True,
#                                    max_features = 'sqrt')
#     # Fit on training data
#     model.fit(X_train, y_train)


#     # Actual class predictions
#     rf_predictions = model.predict(X_test)
#     # Probabilities for each class
#     rf_probs = model.predict_proba(X_test)[:, 1]

#     # Calculate roc auc
#     roc_value = roc_auc_score(y_test, rf_probs)
#     return roc_value, model

    # define the keras model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit the keras model on the dataset
    model.fit(X_train, y_train, epochs=150, batch_size=10)

    # evaluate the keras model
    _, accuracy = model.evaluate(X_test, y_test)
    print('Accuracy: %.2f' % (accuracy*100))
    
    return model

In [137]:
df = df1v2

year = 2014

df['Target'] = [int(x) for x in df['Target']]

# remove outliers
for column in df.columns:
    df[(np.abs(stats.zscore(df[column])) < 2)]

# normalize values
transformer = Normalizer().fit(df)
df = transformer.transform(df)

y = df['Target']

data = df.drop(['Target', str(year+1) + ' PRICE VAR [%]', str(year+2) + ' PRICE VAR [%]'], axis=1)
feature_names = data.columns

X = df[feature_names]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

print(X_train.shape)

# define the keras model
model = Sequential()
model.add(Dense(10, input_dim=20, activation='relu'))
# model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='softmax'))

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150)

# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [129]:
df

,2014 Revenue Growth,Net Income,EPS,EBITDA Margin,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,Debt to Equity,ROIC,2015 PRICE VAR [%],...,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities
0,-0.0713,1.164300e+10,4.1900,0.2470,2.8583,18.7566,21.0348,0.5061,0.0753,-9.323276,...,0,0,1,0,0,0,0,0,0,0
1,1.1737,1.358227e+08,0.2396,0.0107,0.0443,81.5526,1.3589,1.5093,0.0000,-25.512193,...,0,0,1,0,0,0,0,0,0,0
2,0.0182,1.519000e+09,1.4700,0.0450,0.1858,12.0340,14.6302,2.1007,0.0859,33.118297,...,0,0,1,0,0,0,0,0,0,0
3,0.0053,2.494140e+08,4.6000,0.0420,0.2491,28.6087,17.2736,-2.7237,0.1062,2.752291,...,0,0,1,0,0,0,0,0,0,0
4,0.0076,1.824400e+09,2.9000,0.2010,1.8610,18.7034,17.6902,1.3445,0.1041,12.897715,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,0.0553,3.984000e+06,0.5600,0.0810,0.5289,16.7321,11.2440,0.2524,0.1048,-2.453386,...,0,0,0,0,0,0,0,0,1,0
2635,0.1028,-8.593600e+04,-0.0400,-0.0010,0.1216,0.0000,12.2691,0.0000,0.0035,29.362884,...,0,0,0,0,0,0,0,0,1,0
2636,-0.1019,1.306200e+07,0.8800,0.1360,1.2131,14.3409,0.0000,0.0279,0.0000,-31.167763,...,0,0,0,0,0,0,0,0,1,0
2637,0.1344,5.760000e+06,1.2400,0.0250,0.2476,13.8790,14.7618,0.0000,0.3154,7.779579,...,0,0,0,0,0,0,0,0,1,0
